# 🔬 Blood Group Detection from Fingerprint using Deep Learning
Organized Google Colab Notebook (Training Once, Then Just Load)

In [ ]:
# Part 1: Setup & Mount Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
import pickle

Mounted at /content/drive


In [ ]:
#  Part 2: Paths & Constants
img_width, img_height = 224, 224
batch_size = 32

data_dir = "/content/drive/My Drive/dataset_blood_group"
model_path = "/content/drive/My Drive/blood_group_detector.h5"
encoder_path = "/content/drive/My Drive/label_encoder.pkl"
checkpoint_path = "/content/drive/My Drive/checkpoints/cp-{epoch:02d}.weights.h5"

In [ ]:
#  Part 3: Load or Train Model
if os.path.exists(model_path):
    model = load_model(model_path)
    with open(encoder_path, 'rb') as f:
        encoder = pickle.load(f)
    print(" Loaded saved model and encoder.")
else:
    print(" Training model...")

    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    classes = train_generator.class_indices.keys()
    encoder = LabelEncoder()
    encoder.fit(list(classes))
    with open(encoder_path, 'wb') as f:
        pickle.dump(encoder, f)

    model = Sequential([
        tf.keras.Input(shape=(img_width, img_height, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(classes), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,
        verbose=1
    )

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10,
        callbacks=[checkpoint_cb]
    )

    model.save(model_path)
    print(" Model trained and saved.")

 Loaded saved model and encoder.


In [ ]:
#  Part 4: Predict from Image
def predict_blood_group(image_path):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    blood_group = encoder.inverse_transform([predicted_class])[0]
    return blood_group

# Example
sample_path = "cluster_5_54.BMP"  # Change this to your image
print("Predicted Blood Group:", predict_blood_group(sample_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted Blood Group: AB-
